In [23]:
#i want to train Bayesian neural network to predict the value of the cars, so I have a lot of data which is categorial and numerical, I want make model is up-to-date so after training I want to save the model and use it in the future, so I want to know how to save the model and load it again to use it in the future and I will give extra new data before using it to predict the value of the car.

In [24]:
#download the data from the mongodb database
import pymongo
import pandas as pd
import numpy as np
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["endterm"]
mycol = mydb["cars"]
mylist = list(mycol.find())
df = pd.DataFrame(mylist)
df = df.drop(columns=['_id'])
df.head()

,brand,model,year,price,city,generation,body_type,engine_volume,transmission,drive,steering_wheel,custom_cleared,mileage,color,VIN,Наличие
0,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3 (бензин),автомат,задний привод,слева,True,NaN,NaN,NaN,NaN
1,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3 (бензин),автомат,задний привод,слева,True,NaN,NaN,NaN,NaN
2,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8 (бензин),механика,передний привод,слева,True,245 555 км,серый металлик,11122233*44****66,NaN
3,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8 (бензин),механика,передний привод,слева,True,245 555 км,серый металлик,11122233*44****66,NaN
4,ВАЗ (Lada),Priora 2170 (седан),2010,2800000,Костанай,2007 - 2015 1 поколение,седан,1.6 (бензин),механика,передний привод,слева,True,189 000 км,зеленый металлик,NaN,NaN


In [25]:
#I want to bayesian neural network to predict the value of the cars, so I have a lot of data which is categorial and numerical
#if mileage is nan I want to replace it with the 10000*(2023-year)
df['mileage'] = df['mileage'].fillna(10000*(2023-df['year']))
#if color is nan I want to replace it with the "не указан"
df['color'] = df['color'].fillna("не указан")
#if VIN is nan I want to replace it with the "не указан"
df['VIN'] = df['VIN'].fillna("не указан")
#if 'Наличие' is nan I want to replace it with the True, else False
df['Наличие'] = df['Наличие'].fillna(True)
df['Наличие'] = df['Наличие'].replace('На заказ', False)
df.head()

,brand,model,year,price,city,generation,body_type,engine_volume,transmission,drive,steering_wheel,custom_cleared,mileage,color,VIN,Наличие
0,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3 (бензин),автомат,задний привод,слева,True,320000,не указан,не указан,True
1,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3 (бензин),автомат,задний привод,слева,True,320000,не указан,не указан,True
2,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8 (бензин),механика,передний привод,слева,True,245 555 км,серый металлик,11122233*44****66,True
3,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8 (бензин),механика,передний привод,слева,True,245 555 км,серый металлик,11122233*44****66,True
4,ВАЗ (Lada),Priora 2170 (седан),2010,2800000,Костанай,2007 - 2015 1 поколение,седан,1.6 (бензин),механика,передний привод,слева,True,189 000 км,зеленый металлик,не указан,True


In [26]:
#converting categorical data to numerical data, we save all key->values to use them later
#make dictionary for every unique color
import json

def make_dict(arr):
    dic = {}
    for i in range(len(arr)):
        dic[arr[i]] = i
    return dic
def save_dict(arr, name):
    dic = make_dict(arr)
    name1='dictionaries/'+name+'.json'
    with open(name1, 'w', encoding="utf-8") as fp:
        json.dump(dic, fp)
# make list of all categorical columns
cat_cols = ['color', 'model', 'brand', 'transmission', 'generation','body_type','steering_wheel', 'city','drive']
for i in cat_cols:
    save_dict(df[i].unique(), i)


In [27]:
#if engine_volume has "газ-бензин" make isPetrol=False, else isPetrol=True
isPetrol = []
for i in range(len(df)):
    if df.engine_volume[i] == 'газ-бензин':
        isPetrol.append(False)
    else:
        isPetrol.append(True)
df['isPetrol'] = isPetrol
#delete all signs in engine_volume except numbers
import re
engine_volume = []
for i in range(len(df)):
    a = df.engine_volume[i]
    engine_volume.append(re.sub('[^\d\.]', '', a))

df['engine_volume'] = engine_volume

In [28]:
mileage = []
for i in range(len(df)):
    a = str(df.mileage[i])
    mileage.append(re.sub('[^\d\.]', '', a))

mileage= [int(i) for i in mileage]
df['mileage'] = mileage
df.head()

,brand,model,year,price,city,generation,body_type,engine_volume,transmission,drive,steering_wheel,custom_cleared,mileage,color,VIN,Наличие,isPetrol
0,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3,автомат,задний привод,слева,True,320000,не указан,не указан,True,True
1,Mercedes-Benz,E 230,1991,1500000,Алматы,1987 - 1993 W124 рестайлинг,седан,2.3,автомат,задний привод,слева,True,320000,не указан,не указан,True,True
2,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8,механика,передний привод,слева,True,245555,серый металлик,11122233*44****66,True,True
3,Volkswagen,Passat,1992,1800000,Шымкент,1988 - 1993 B3,универсал,1.8,механика,передний привод,слева,True,245555,серый металлик,11122233*44****66,True,True
4,ВАЗ (Lada),Priora 2170 (седан),2010,2800000,Костанай,2007 - 2015 1 поколение,седан,1.6,механика,передний привод,слева,True,189000,зеленый металлик,не указан,True,True


In [29]:
#convert all categorical data to numerical data
import json
def load_dict(name):
    name1='dictionaries/'+name+'.json'
    with open(name1, 'r', encoding="utf-8") as fp:
        dic = json.load(fp)
    return dic
#change all categorical data to numerical
for i in cat_cols:
    dic = load_dict(i)
    df[i] = df[i].map(dic)

In [30]:
df=df.drop(columns=['VIN'])
df["engine_volume"] = df["engine_volume"].astype(float)
df.head()


,brand,model,year,price,city,generation,body_type,engine_volume,transmission,drive,steering_wheel,custom_cleared,mileage,color,Наличие,isPetrol
0,0,0,1991,1500000,0,0,0,2.3,0,0,0,True,320000,0,True,True
1,0,0,1991,1500000,0,0,0,2.3,0,0,0,True,320000,0,True,True
2,1,1,1992,1800000,1,1,1,1.8,1,1,0,True,245555,1,True,True
3,1,1,1992,1800000,1,1,1,1.8,1,1,0,True,245555,1,True,True
4,2,2,2010,2800000,2,2,0,1.6,1,1,0,True,189000,2,True,True


In [31]:
#split the data to train and test
from sklearn.model_selection import train_test_split
X = df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
#make bayesian neural network model and train it with the train data and plot the loss and accuracy and then save the model to use it in the future
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
xg_reg = xgb.XGBRegressor(objective="reg:squarederror", colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)
xg_reg.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = xg_reg.predict(X_test)

# Evaluate model performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 3913673.116567


In [33]:
#I want to ouput the R2 score of the model
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9574911785090534

In [34]:
#I want to output the mean absolute error of the model
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

1756800.4510135136

In [35]:
#I want to output the mean squared error of the model
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

15316837263340.656

In [36]:
#I want to output the median absolute error of the model
from sklearn.metrics import median_absolute_error
median_absolute_error(y_test, y_pred)

776056.0

In [37]:
#I want to output the explained variance score of the model
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_pred)

0.9601184398534589

In [38]:
#I want to output the max error of the model
from sklearn.metrics import max_error
max_error(y_test, y_pred)

18476560.0

In [39]:
#save the model
import joblib
joblib.dump(xg_reg, 'xg_reg.pkl')

['xg_reg.pkl']

In [40]:
#load the model
loaded_model = joblib.load('xg_reg.pkl')

In [49]:
#I want to take random value from X_test and predict the price of the car
import random
random.seed(35)
a = random.randint(0, len(X_test))
print(a)

70


In [50]:
loaded_model.predict(X_test.iloc[a].values.reshape(1, -1))

array([5716669.5], dtype=float32)

In [51]:
y_test.iloc[a]

4800000